In [1]:
from datetime import datetime
import numpy as np
import time
import os
import tabula


DB_TRANSACTIONS = os.environ['NOTION_MONEY_TRANSACTIONS']  # Replace with your database ID
DB_CSV = os.environ['NOTION_MONEY_CSV']  # Replace with your database ID
NOTION_API_KEY = os.environ['NOTION_TOKEN']  # Replace with your database ID

KeyError: 'NOTION_MONEY_TRANSACTIONS'

In [ ]:
from notion_client import Client
from pprint import pprint
import requests
import json
import pandas as pd


headers = {
    "Authorization": "Bearer " + NOTION_API_KEY,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

def get_pages(num_pages=None):
    """
    If num_pages is None, get all pages, otherwise just the defined number.
    """
    url = f"https://api.notion.com/v1/databases/{DB_CSV}/query"

    get_all = num_pages is None
    page_size = 100 if get_all else num_pages

    filter = {
        "filter": {
            "property": "Added",
            "checkbox": {
            "equals": False
            }
        },
    }
    response = requests.post(url, json=filter, headers=headers)

    data = response.json()

    results = data["results"]
    while data["has_more"] and get_all:
        payload = {"page_size": page_size, "start_cursor": data["next_cursor"]}
        url = f"https://api.notion.com/v1/databases/{DB_CSV}/query"
        response = requests.post(url, json=filter, headers=headers)
        data = response.json()
        results.extend(data["results"])

    return results

notion = Client(auth=NOTION_API_KEY)
list_users_response = notion.users.list()

results = get_pages()

In [ ]:
list_n26 = []
list_buddy = []

for item in results:
    # url = item["properties"]["CSV"]["files"][0]["file"]["url"]
    try:
        if item["properties"]["Name"]["title"][0]["text"]["content"].find("Buddy") != -1:
            list_buddy.append(
                {   
                    "id": item["id"],
                    "url": item["properties"]["CSV"]["files"][0]["file"]["url"]
                }
            )
        elif item["properties"]["Name"]["title"][0]["text"]["content"].find("N26") != -1:
            list_n26.append(
                {   
                    "id": item["id"],
                    "url": item["properties"]["CSV"]["files"][0]["file"]["url"]
                }
            )
    except:
        print("No File Attached")

In [ ]:
def create_item(title, description='', date='', account ='', amount='', category=''):
    if not category:
        category = 'Other'
    if description == 'nan':
        description = ''
    url = f'https://api.notion.com/v1/pages'
    # Prepare the payload for the new task
    payload = {
        'parent': {
            'database_id': f'{DB_TRANSACTIONS}'
        },
        
        'properties': {
            'Name': {
                'title': [{
                    'text': {
                        'content': title
                    }
                }]
            },
            'Description': {
                'rich_text': [{
                    'text': {
                        'content': str(description)
                    }
                }]
            },
            'Transaction Date': {
                'date': {
                    'start': date
                }
            },
            'Account': {
                'select': {
                    'name': account
                }
            },
            'Price': {
                'number': float(amount)
            },
            'Category': {
                "select": {
                    "name": category
                    },
            },
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        print(f"Task '{title}' created successfully.")
        return 1
    else:
        print(f"Error creating task: {response.status_code} {response.text}")
        return 0

# Buddybank.pdf
Tested on Q2 24 with big number of element.

### FD
Add partial sum verification as for Buddybank Lite 

In [ ]:
def buddy_to_notion(df):
    try:
        df["Entrate"] = (df['Entrate'].str.replace('.', '').str.replace(',', '.').astype(float))
    except:
        pass
    
    try:
        df["Uscite"] = (df['Uscite'].str.replace('.', '').str.replace(',', '.').astype(float))
    except:
        pass

    for index, row in df.iterrows():
        val_date_str = row["Valuta"]

        try:
            val_date = datetime.strptime(val_date_str, "%Y-%m-%d")
        except:
            try:
                val_date = datetime.strptime(val_date_str, "%d.%m.%y")
            except:
                print(f"Row is not a valid one, shit")
                continue

        try:
            title = row["Descrizione"].split("*")[2]
        except:
            title = row["Descrizione"]

        account = "Buddybank"
        
        if not np.isnan(row["Entrate"]): 
            amount = row["Entrate"]
        else: 
            amount = - abs(row["Uscite"])
        
        create_item(title=title,
                    date=val_date.strftime("%Y-%m-%d"),
                    account=account,
                    amount=amount)
    else:
            print("Execution Complete")
            return

In [ ]:
for list_item in list_buddy:
    try: 
        tables = tabula.read_pdf(list_item["url"], pages="all", multiple_tables=True, lattice=True)
        df_temp = pd.concat(tables)

        df_temp = df_temp[1:] #take the data less the header row
        # df_temp.columns = new_header #set the header row as the df header
        df_temp = df_temp[df_temp.Valuta != 'Valuta']
        df_temp.dropna(how='all', axis=0, inplace=True)
        df_temp.dropna(how='all', axis=1, inplace=True)
        df_temp = df_temp[1:] #take the data less the header row
        df_temp = df_temp[:-2].reindex()

        for index, item in df_temp.iterrows():
            if item["Descrizione"].find("EUR") != -1:
                item["Descrizione"] = item["Descrizione"][(item["Descrizione"].find("EUR"))+9:].strip()
                print(item["Descrizione"].strip())
            elif item["Descrizione"].find("AUD") != -1:
                item["Descrizione"] = item["Descrizione"][(item["Descrizione"].find("AUD"))+9:].strip()
            else:
                pass
        
        buddy_to_notion(df_temp)
    except:
        print(f"Error with {item}")
        print("no")
    else:
        notion.pages.update(
            list_item["id"],
            properties={
                "Added": {
                    "checkbox": True
                }
            },
        )

# N26.pdf
Last run in Sept 2024. Verify after usage

In [ ]:
import tabula

def N26_to_notion(df):
    
    for index, rows in df.iterrows():
        date_str = rows["Booking Date"]
        date = datetime.strptime(date_str, "%d.%m.%Y")
        if not create_item( title=f'{rows["Description"]}',
                            date=f'{date.strftime("%Y-%m-%d")}',
                            account=f'{"N26"}',
                            amount=f'{rows["Amount"]}',
                            description='',
                            category='Other'):
            print("Error in execution, correct the configuration and start again")
            break
    else:
        print("Execution Complete")

In [ ]:
header = ['Description', 'Booking Date', 'Amount']

for list_item in list_n26:
    try: 
        tables = tabula.read_pdf(list_item["url"], pages="all", multiple_tables=False, lattice=False)

        df_temp = tables[0]
        df_temp = pd.DataFrame(df_temp.values[0:], columns=header)
        restore_row = tables[0].columns.to_list()
        df_temp = pd.concat([df_temp, pd.DataFrame([tables[0].columns.to_list()], columns=header)], ignore_index=True, axis=0) # does not save changes to the original dataframe
        df_temp["Amount"] = df_temp["Amount"].str.replace('€','').str.replace('.','').str.replace(',','.').astype(float)
        df_temp = df_temp[df_temp['Amount'].notna()]
        
        N26_to_notion(df_temp)
        
    except:
        print("Brutta madonna")
    else:
        notion.pages.update(
            item["id"],
            properties={
                "Added": {
                    "checkbox": True
                }
            },
        )

In [ ]:
tables = tabula.read_pdf(list_item["url"], pages="all", multiple_tables=False, lattice=False)

df_temp = tables[0]
df_temp = pd.DataFrame(df_temp.values[0:], columns=header)
restore_row = tables[0].columns.to_list()
df_temp = pd.concat([df_temp, pd.DataFrame([tables[0].columns.to_list()], columns=header)], ignore_index=True, axis=0) # does not save changes to the original dataframe
df_temp["Amount"] = df_temp["Amount"].str.replace('€','').str.replace('.','').str.replace(',','.').astype(float)
df_temp = df_temp[df_temp['Amount'].notna()]

N26_to_notion(df_temp)

# BuddybankMonthly.pdf (Lite)

In [6]:
YEAR = 2025

import os
import re
import unicodedata
import pandas as pd
import numpy as np
from datetime import datetime
from pypdf import PdfReader

# === Funzione per pulire la descrizione ===
def pulisci_descrizione(testo):
    if not isinstance(testo, str):
        return ""

    testo = testo.strip()

    # Bonifico in entrata: "BONIFICO A VOSTRO FAVORE ... DA:"
    if "BONIFICO A VOSTRO FAVORE" in testo.upper() and "DA:" in testo.upper():
        m = re.search(r"DA:\s*([^,]+)", testo, re.IGNORECASE)
        if m:
            return m.group(1).strip()

    # Bonifico in uscita: "BONIFICO SEPA A:"
    if "BONIFICO SEPA A:" in testo.upper():
        m = re.search(r"A:\s*([^P]+)", testo, re.IGNORECASE)
        if m:
            return m.group(1).strip()

    # Pagamenti con carta, contactless, Apple Pay, ecc.
    if "PAGAMENTO" in testo.upper():
        m = re.search(r"EUR [\d,.]+\s*(.*)", testo, re.IGNORECASE)
        if m:
            beneficiario = m.group(1)
            beneficiario = re.sub(r"\b\d+\b", "", beneficiario)  # rimuove numeri
            beneficiario = re.sub(r"\s{2,}", " ", beneficiario)   # rimuove spazi doppi
            return beneficiario.strip()

    # Addebiti SEPA o PayPal
    if "SEPA" in testo.upper() or "PAYPAL" in testo.upper():
        m = re.search(r"PayPal[^\s,]*", testo, re.IGNORECASE)
        if m:
            return m.group(0).strip()
        m = re.search(r"da\s+([A-Za-z\s\.&]+)\s+mandato", testo, re.IGNORECASE)
        if m:
            return m.group(1).strip()

    # Fallback: testo pulito senza simboli eccessivi
    testo = re.sub(r"\s{2,}", " ", testo)
    return testo.strip()

# Dizionario mesi italiani → inglesi
it_to_eng_dict = {
    "gennaio": "Jan",
    "febbraio": "Feb",
    "marzo": "Mar",
    "maggio": "May",
    "giugno": "Jun",
    "luglio": "Jul",
    "agosto": "Aug",
    "settembre": "Sep",
    "ottobre": "Oct",
    "novembre": "Nov",
    "dicembre": "Dec"
}

# Directory contenente i PDF
pdf_dir = r"C:\Users\loren\OneDrive\Desktop\Areas\Finanza"
pdf_files = [
    f for f in os.listdir(pdf_dir)
    if f.lower().endswith(".pdf") and "buddy" in f.lower() and "lite" in f.lower()
]

# Regex e funzioni di supporto
regex_list = [r"\b\d\d\s", r"\b\d\s", r"\b\d"]

def remove_elements_at_indices(test_list, idx_list):
    if not idx_list:
        return test_list
    first_idx = idx_list[0]
    rest_of_indices = idx_list[1:]
    sub_list = remove_elements_at_indices(test_list, rest_of_indices)
    sub_list.pop(first_idx)
    return sub_list

# Inizio elaborazione
print(f"\nTrovati {len(pdf_files)} file PDF contenenti 'Buddy' e 'Lite':")
for f in pdf_files:
    print(f"  - {f}")
print("\nInizio elaborazione...\n")

# Processamento PDF
for file_name in pdf_files:
    print("=" * 70)
    print(f"FILE: {file_name}")
    print("-" * 70)

    df_temp = pd.DataFrame(columns=["Data", "Beneficiario", "Importo"])
    file_path = os.path.join(pdf_dir, file_name)
    reader = PdfReader(file_path)

    page_lines = []
    for page in reader.pages:
        text = page.extract_text()
        lines = text.splitlines()
        page_lines.extend(lines)

    # Rimozione numeri di pagina
    idx_list = [index for (index, item) in enumerate(page_lines)
                if re.match(r"(?<!\S)\d(?!\S)(?!.)", item)]
    page_lines = remove_elements_at_indices(page_lines, idx_list)

    # Individuazione inizio dati
    dates_indexes = [index for (index, item) in enumerate(page_lines)
                     if re.match(regex_list[0], item) or re.match(regex_list[1], item)]
    if not dates_indexes:
        print("Nessuna data trovata. File saltato.")
        continue

    page_lines = page_lines[dates_indexes[0]:]
    dates_indexes = [index for (index, item) in enumerate(page_lines)
                     if re.match(regex_list[0], item) or re.match(regex_list[1], item)]

    buffer = ""
    skip = False

    for index, line in enumerate(page_lines):
        if index in dates_indexes:
            date = page_lines[index]
            continue

        if (index < len(page_lines)-1 and page_lines[index] == page_lines[index+1]) or skip:
            skip = False
            continue

        if unicodedata.lookup("EURO SIGN") not in page_lines[index] and (index not in dates_indexes):
            buffer += page_lines[index]
            continue

        if unicodedata.lookup("EURO SIGN") in page_lines[index]:
            new_row = buffer + page_lines[index]
            buffer = ""

        try:
            money = re.search(r"(-\d+\,?\d*\s\€|\d+\,?\d*\s\€)", new_row)
            if not money:
                continue

            descrizione = new_row[:money.start()]
            val_date_str = re.sub(r'\b' + r'|\b'.join(it_to_eng_dict) + r'\b',
                                  lambda m: it_to_eng_dict.get(m.group(), m.group()), date)
            val_date_str = val_date_str + f" {YEAR}"
            val_date = datetime.strptime(val_date_str, "%d %b %Y")
            money_val = float(money.group().replace('€', '').replace('.', '').replace(',', '.'))

            if money_val < 0:
                new_df_row = {"Data": val_date.strftime("%m/%d/%Y"),
                              "Beneficiario": descrizione, "Importo": money_val}
            else:
                new_df_row = {"Data": val_date.strftime("%m/%d/%Y"),
                              "Beneficiario": descrizione, "Importo": money_val}

            df_temp = pd.concat([df_temp, pd.DataFrame([new_df_row])], ignore_index=True)

        except Exception:
            continue

    # Verifica somme
    df_temp["Importo"] = df_temp["Importo"].replace(r'^\s*$', np.nan, regex=True)

    try:
        total_sum = df_temp["Importo"].iloc[-1]
        partial_sum = df_temp["Importo"][:-1].sum()
        if total_sum != round(partial_sum, 2):
            print("AVVISO: somma parziale diversa dal totale indicato.")
            print(f"  Parziale: {partial_sum:.2f}")
            print(f"  Totale  : {total_sum}")
        else:
            print("Somme verificate correttamente.")
    except Exception:
        print("Impossibile verificare le somme (dati non completi).")

    # Pulizia descrizioni
    df_temp["Beneficiario"] = df_temp["Beneficiario"].apply(pulisci_descrizione)

    # Salvataggio
    output_path = os.path.join(pdf_dir, f"{file_name[:-4]}.csv")
    df_temp.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"Transazioni estratte: {len(df_temp)}")
    print(f"File CSV generato: {output_path}")
    print("=" * 70 + "\n")

print("Elaborazione completata.")



Trovati 1 file PDF contenenti 'Buddy' e 'Lite':
  - BuddybankLite_October.pdf

Inizio elaborazione...

FILE: BuddybankLite_October.pdf
----------------------------------------------------------------------


C:\Users\loren\AppData\Local\Temp\ipykernel_2744\4136249107.py:163: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_temp = pd.concat([df_temp, pd.DataFrame([new_df_row])], ignore_index=True)


AVVISO: somma parziale diversa dal totale indicato.
  Parziale: 1218.87
  Totale  : 460.42
Transazioni estratte: 569
File CSV generato: C:\Users\loren\OneDrive\Desktop\Areas\Finanza\BuddybankLite_October.csv

Elaborazione completata.


# Trade Republic

In [23]:
YEAR = 2025

import os
import re
import unicodedata
import pandas as pd
import numpy as np
from datetime import datetime
from pypdf import PdfReader
import pdfplumber
import pandas as pd

# === Funzione per pulire la descrizione ===
def pulisci_descrizione(testo):
    if not isinstance(testo, str):
        return ""

    testo = testo.strip()

    # Bonifico in entrata: "BONIFICO A VOSTRO FAVORE ... DA:"
    if "BONIFICO A VOSTRO FAVORE" in testo.upper() and "DA:" in testo.upper():
        m = re.search(r"DA:\s*([^,]+)", testo, re.IGNORECASE)
        if m:
            return m.group(1).strip()

    # Bonifico in uscita: "BONIFICO SEPA A:"
    if "BONIFICO SEPA A:" in testo.upper():
        m = re.search(r"A:\s*([^P]+)", testo, re.IGNORECASE)
        if m:
            return m.group(1).strip()

    # Pagamenti con carta, contactless, Apple Pay, ecc.
    if "PAGAMENTO" in testo.upper():
        m = re.search(r"EUR [\d,.]+\s*(.*)", testo, re.IGNORECASE)
        if m:
            beneficiario = m.group(1)
            beneficiario = re.sub(r"\b\d+\b", "", beneficiario)  # rimuove numeri
            beneficiario = re.sub(r"\s{2,}", " ", beneficiario)   # rimuove spazi doppi
            return beneficiario.strip()

    # Addebiti SEPA o PayPal
    if "SEPA" in testo.upper() or "PAYPAL" in testo.upper():
        m = re.search(r"PayPal[^\s,]*", testo, re.IGNORECASE)
        if m:
            return m.group(0).strip()
        m = re.search(r"da\s+([A-Za-z\s\.&]+)\s+mandato", testo, re.IGNORECASE)
        if m:
            return m.group(1).strip()

    # Fallback: testo pulito senza simboli eccessivi
    testo = re.sub(r"\s{2,}", " ", testo)
    return testo.strip()


def clean_page_lines(page_lines):
    header_re = re.compile(r"DATA\s+TIPO\s+DESCRIZIONE\s+IN\s+ENTRATA\s+IN\s+USCITA\s+SALDO", re.IGNORECASE)
    euro_re = re.compile(r"€")

    # 1. Trova tutti gli indici delle intestazioni
    header_indices = [i for i, l in enumerate(page_lines) if header_re.search(l)]
    if not header_indices:
        return page_lines[:]  # nessuna intestazione trovata

    # 2. Elimina righe prima della prima intestazione
    first_header = header_indices[0]
    lines = page_lines[first_header:]
    rel_header_indices = [i - first_header for i in header_indices if i >= first_header]
    n = len(lines)

    cleaned = []
    pos = 0

    for h_idx, header_pos in enumerate(rel_header_indices):
        # Definisci il blocco corrente (tra intestazioni)
        next_header = rel_header_indices[h_idx + 1] if h_idx + 1 < len(rel_header_indices) else n

        # Trova l’ultima riga con "€" prima della prossima intestazione
        last_euro = None
        for i in range(header_pos + 1, next_header):
            if euro_re.search(lines[i]):
                last_euro = i

        # Mantieni solo le righe dopo l’intestazione e fino all’ultima con €
        if last_euro is not None:
            cleaned.extend(lines[header_pos + 1:last_euro + 1])

    # 3. Dopo l’ultima intestazione, controlla eventuali righe residue con “€” (es. fine documento)
    if rel_header_indices and rel_header_indices[-1] < n:
        tail_start = rel_header_indices[-1]
        last_euro = None
        for i in range(tail_start + 1, n):
            if euro_re.search(lines[i]):
                last_euro = i
        if last_euro is not None:
            cleaned.extend(lines[tail_start + 1:last_euro + 1])

    return cleaned

def parse_transactions(lines):
    """
    lines: lista di stringhe (righe estratte dal PDF)
    restituisce: DataFrame con colonne ['Data','Beneficiario','Importo','Saldo']
    """
    # pattern
    day_pattern = re.compile(r"^\d{1,2}$")
    month_pattern = re.compile(r"^[A-Za-zÀ-ÖØ-öø-ÿ]+$")   # mese (es. mag, nov, Aprile)
    day_month_pattern = re.compile(r"^\d{1,2}\s+[A-Za-zÀ-ÖØ-öø-ÿ]+$")  # "01 mag"
    year_pattern = re.compile(r"^\d{4}$")
    header_pattern = re.compile(r"DATA\s+TIPO\s+DESCRIZIONE\s+IN\s+ENTRATA\s+IN\s+USCITA\s+SALDO", re.IGNORECASE)
    panorama_pattern = re.compile(r"PANORAMICA", re.IGNORECASE)
    euro_pattern = re.compile(r"([\d\.\,]+)\s*€")

    records = []
    i = 0
    n = len(lines)

    # opzionale: elimina tutto prima della prima header se presente
    first_header_idx = next((idx for idx, l in enumerate(lines) if header_pattern.search(l)), None)
    if first_header_idx is not None:
        lines = lines[first_header_idx:] 
        n = len(lines)

    while i < n:
        # prova a riconoscere una data a partire da lines[i]
        date_str = None
        consumed = 0

        s = lines[i].strip()
        # caso 1: "01 mag" in una riga
        if day_month_pattern.match(s):
            date_str = s
            consumed = 1
        else:
            # caso 2: giorno su una riga, mese su successiva, anno su successiva (o mese+anno)
            if day_pattern.match(s) and i+1 < n and month_pattern.match(lines[i+1].strip()):
                # giorno + mese
                day = s
                month = lines[i+1].strip()
                # anno possibile
                if i+2 < n and year_pattern.match(lines[i+2].strip()):
                    year = lines[i+2].strip()
                    date_str = f"{day} {month} {year}"
                    consumed = 3
                else:
                    date_str = f"{day} {month}"
                    consumed = 2
            # caso 3: giorno e anno separate (meno probabile), oppure riga già "01 apr 2025" (coperto da day_month_pattern? no)
            elif day_pattern.match(s) and i+1 < n and year_pattern.match(lines[i+1].strip()):
                date_str = f"{s} {lines[i+1].strip()}"
                consumed = 2
            # caso 4: riga già "01 apr 2025" (giorno mese anno in una riga)
            elif re.match(r"^\d{1,2}\s+[A-Za-zÀ-ÖØ-öø-ÿ]+\s+\d{4}$", s):
                date_str = s
                consumed = 1
            # caso 5: riga tipo "01 apr" / next line contains "2025 Trasferimento..." -> treat as date if next starts with year
            elif day_month_pattern.match(s) and i+1 < n and year_pattern.match(lines[i+1].strip().split()[0]):
                date_str = s + " " + lines[i+1].strip().split()[0]
                consumed = 2

        if date_str is None:
            # non è una data: avanza
            i += 1
            continue

        # avanzare oltre le righe consumate della data
        i += consumed
        if i >= n:
            break

        # ora raggruppa le righe fino a raggiungere il saldo (ultimo € della transazione)
        full_parts = []
        last_euro_count = 0
        safety_counter = 0
        while i < n:
            line = lines[i].strip()
            # fermati se incontri una header o PANORAMICA (è probabile che la transazione sia incompleta)
            if header_pattern.search(line) or panorama_pattern.search(line):
                break

            full_parts.append(line)
            last_euro_count = sum(1 for part in full_parts if "€" in part)
            i += 1
            safety_counter += 1

            # condizione di stop robusta:
            # - se abbiamo almeno un '€' e la riga corrente contiene '€' e la riga successiva è header/date/PANORAMICA, oppure
            # - se abbiamo >=2 simboli € (importo + saldo),
            # - oppure se raggiungiamo la fine
            next_line = lines[i].strip() if i < n else ""
            if last_euro_count >= 2:
                break
            if "€" in full_parts[-1] and ( header_pattern.search(next_line) or panorama_pattern.search(next_line) or day_pattern.match(next_line) or day_month_pattern.match(next_line) ):
                # probabilmente saldo in questa riga
                break

            # safety: non lasciare mai loop infinito
            if safety_counter > 2000:
                break

        # costruisci la descrizione completa
        full_desc = " ".join([p for p in full_parts if p])
        # estrai tutti gli importi trovati in ordine
        euro_matches = euro_pattern.findall(full_desc)

        # normalizza importo e saldo (primo e ultimo match se disponibili)
        amount = None
        balance = None
        if euro_matches:
            if len(euro_matches) == 1:
                # potrebbe essere solo importo (o solo saldo): assumiamo importo e nessun saldo
                amount = euro_matches[0].replace('.', '').replace(',', '.')
                balance = None
            else:
                amount = euro_matches[0].replace('.', '').replace(',', '.')
                balance = euro_matches[-1].replace('.', '').replace(',', '.')

        # beneficiary: la descrizione senza gli importi
        beneficiary = euro_pattern.sub("", full_desc).strip()
        # se la descrizione inizia con un anno o verbo ripulire eventuale anno residuo
        beneficiary = re.sub(r"^\d{4}\b", "", beneficiary).strip()

        # salva record
        try:
            amount_f = float(amount) if amount is not None else None
        except:
            amount_f = None
        try:
            balance_f = float(balance) if balance is not None else None
        except:
            balance_f = None

        records.append({
            "Data": date_str,
            "Beneficiario": beneficiary,
            "Importo": amount_f,
            "Saldo": balance_f
        })

    # crea DataFrame
    df = pd.DataFrame(records, columns=["Data", "Beneficiario", "Importo", "Saldo"])
    return df

MONTHS_IT = {
    "gen": "january",
    "feb": "february",
    "mar": "march",
    "apr": "april",
    "mag": "may",
    "giu": "june",
    "lug": "july",
    "ago": "august",
    "set": "september",
    "ott": "october",
    "nov": "november",
    "dic": "december",
}

def normalize_date(date_str):
    """
    Converte 'dd mmm' o 'dd mmm YYYY' con mesi italiani
    nel formato 'mm/dd/YYYY'. Aggiunge 2025 se l'anno manca.
    """
    if not isinstance(date_str, str):
        return None

    parts = date_str.strip().split()

    if len(parts) == 2:
        day, month = parts
        year = "2025"
    elif len(parts) == 3:
        day, month, year = parts
    else:
        return None

    month = month.lower()

    # normalizza mese breve → mese esteso italiano
    if month in MONTHS_IT:
        month = MONTHS_IT[month]

    # parsing mesi italiani
    try:
        dt = datetime.strptime(f"{day} {month} {year}", "%d %B %Y")
    except ValueError:
        return None

    return dt.strftime("%m/%d/%Y")

def normalize_date_column(df, column_name):
    """
    Applica normalize_date a tutte le celle della colonna.
    Restituisce il DataFrame con la colonna convertita.
    """
    df[column_name] = df[column_name].apply(normalize_date)
    return df

# Directory contenente i PDF
pdf_dir = r"C:\Users\loren\OneDrive\Desktop\Areas\Finanza"
pdf_files = [
    f for f in os.listdir(pdf_dir)
    if f.lower().endswith(".pdf") and "traderepublic" in f.lower()
]

# Regex e funzioni di supporto
regex_list = [r"\b\d\d\s", r"\b\d\s", r"\b\d"]

def remove_elements_at_indices(test_list, idx_list):
    if not idx_list:
        return test_list
    first_idx = idx_list[0]
    rest_of_indices = idx_list[1:]
    sub_list = remove_elements_at_indices(test_list, rest_of_indices)
    sub_list.pop(first_idx)
    return sub_list

def clean_beneficiario(df, column="Beneficiario"):
    """
    Rimuove dal campo Beneficiario prefissi indesiderati.
    Pulizia case-insensitive, elimina spazi extra.
    """
    patterns = [
        r"^transazione con carta\s*",
        r"^pagamento degli interessi\s*",
        r"^premio\s*",
        r"^commercio\s*",
        r"^redditi\s*",
    ]

    combined = re.compile("|".join(patterns), flags=re.IGNORECASE)

    df[column] = (
        df[column]
        .str.replace(combined, "", regex=True)
        .str.strip()
    )

    return df

def apply_import_sign(df, import_col="Importo", saldo_col="Saldo"):
    """
    Determina il segno degli importi confrontando il saldo precedente.
    Se saldo_prev + importo == saldo_corr → importo positivo.
    Altrimenti → negativo.
    Il segno viene aggiunto come stringa: '+...' oppure '-...'.
    """
    
    saldi = df[saldo_col].astype(float).values
    importi = df[import_col].astype(float).values

    signed_values = []

    for i in range(len(df)):
        if i == 0:
            # nessun saldo precedente: non si può dedurre il segno
            signed_values.append(f"{importi[i]:.2f}")
            continue
        
        saldo_prev = saldi[i-1]
        saldo_curr = saldi[i]
        imp = importi[i]

        # confronto con tolleranza per floating point
        if abs((saldo_prev + imp) - saldo_curr) < 0.0001:
            sign = "+"
        else:
            sign = "-"

        signed_values.append(f"{sign}{imp:.2f}")

    df[import_col] = signed_values
    return df

# Inizio elaborazione
print(f"\nTrovati {len(pdf_files)} file PDF contenenti 'TradeRepublic':")
for f in pdf_files:
    print(f"  - {f}")
print("\nInizio elaborazione...\n")

# Processamento PDF
for file_name in pdf_files:
    print("=" * 70)
    print(f"FILE: {file_name}")
    print("-" * 70)

    df_temp = pd.DataFrame(columns=["Data", "Beneficiario", "Importo"])
    file_path = os.path.join(pdf_dir, file_name)
    reader = PdfReader(file_path)

    page_lines = []
    for page in reader.pages:
        text = page.extract_text()
        lines = text.splitlines()
        page_lines.extend(lines)

    cleaned_lines = clean_page_lines(page_lines)

    df_temp = parse_transactions(cleaned_lines)
    df_temp = normalize_date_column(df_temp, "Data")
    df_temp = clean_beneficiario(df_temp, column="Beneficiario")
    df_temp = apply_import_sign(df_temp)

    # Salvataggio
    output_path = os.path.join(pdf_dir, f"{file_name[:-4]}.csv")
    df_temp.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"Transazioni estratte: {len(df_temp)}")
    print(f"File CSV generato: {output_path}")
    print("=" * 70 + "\n")

print("Elaborazione completata.")



Trovati 1 file PDF contenenti 'TradeRepublic':
  - TradeRepublic_October.pdf

Inizio elaborazione...

FILE: TradeRepublic_October.pdf
----------------------------------------------------------------------
Transazioni estratte: 99
File CSV generato: C:\Users\loren\OneDrive\Desktop\Areas\Finanza\TradeRepublic_October.csv

Elaborazione completata.
